# Retrieval Augmented Generation
This notebook provides an introduction to Retrieval Augmented Generation.

For this tutorial, we'll use LlamaIndex, which provides some abstractions over underlying APIs used for building LLM applications. For more about RAG and LlamaIndex's definitions, see [here](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html).

In [14]:
import os
from glob import glob

# We'll load documents that we've already downloaded in the Synthetic Dataset for RAG
data_dir = os.path.join(os.path.abspath(""), ".content/docs")
input_files = glob(os.path.join(data_dir, "*.txt"))
print("Files in folder:")
for f in input_files:
    print(os.path.basename(f))

Files in folder:
1e41916248531ab7c35d0c9895b9e097.txt
1c47198b78d3baab084ebbfd0b5d1e43.txt
dac2964c9d6191656e40e38d6cfa797c.txt
bb56a54b4a7ee19ff73c7fc87b6cc6fa.txt
8e534c9391d545169b1c2ff3432c4b97.txt
0db0d4d2d9d1f862054912669da27ea0.txt
39ee0575b28fb7167abb2755ce57c675.txt
aeae8a7470762d8d7bb42085d56e5c40.txt
c9ccbee73c031d98e8469403f388a4e7.txt
0b4f5ff1e9026497acf63bc503b070be.txt
24e0e416b1e6abb4971d5965ba6c6aeb.txt
f6c56cde7a497644d5cd3a187335ceb8.txt
a872fdc79a7d1bff8c2cbd5583eb4de9.txt
4148ec1d40b273663a2d6ca8bd9b9ac5.txt
ebf98fe7bcedaa0a1085a40346c72d13.txt
ac792786fb9d1ed1cdb868c26c72dec1.txt
2c50ca4fb25df0c314ae76832315081d.txt
c6c7912399e615bd689d0d004ddae408.txt
4f37b66d7e592c35ef062031918b52c4.txt
715dbe32582d64924bfc08a298a90c09.txt
35b7eb0bd521f72d9f72fe60f8acee81.txt
d84394b277b4cd3b684ce126842c8eb4.txt
5298c1b42aa3f0b088f4f0fee71be3a1.txt
57e470c6120ad0023501864f7506050a.txt
e46dc5455452ad047b9332419028605e.txt
36c61e82333cfd0ab0c5f39184a8b47c.txt
930b10d667785c0007828

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

## A simple RAG with LlamaIndex (using defaults)

In [16]:
from llama_index import ServiceContext, SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms import OpenAI

First, build the index from all the documents we have. Using default chunking, and embedding strategies.

In [17]:
service_context = ServiceContext.from_defaults(llm=OpenAI())
documents = SimpleDirectoryReader(input_files=input_files).load_data("*.txt")
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Loading files: 100%|██████████| 50/50 [00:00<00:00, 1626.39file/s]


In [18]:
query_engine = index.as_query_engine()

In [19]:
response = query_engine.query("What are the SI units for speed?")
print(response)

The SI units for speed are meters per second (m/s).


In [20]:
response = query_engine.query("What are the slowest seismic waves?")
print(response)

The slowest seismic waves are the surface waves.


In [21]:
question = "Why do birds lay eggs?"
response = query_engine.query(question)
print("Question:")
print(question)
print("Answer:")
print(str(response))
print("Chunks:")
for node in response.source_nodes:
    print("--------------------------")
    print(str(node.text))
    print("--------------------------")

Question:
Why do birds lay eggs?
Answer:
Birds lay eggs because they are endothermic tetrapod vertebrates. The eggs they lay have hard, calcium carbonate shells and are amniotic, which means they are protected by a membrane. This reproductive strategy allows birds to reproduce and ensure the survival of their offspring.
Chunks:
--------------------------
Birds are endothermic tetrapod vertebrates. They are bipedal, which means they walk on two legs. Birds also lay amniotic eggs, and the eggs have hard, calcium carbonate shells. Although birds are the most recent class of vertebrates to evolve, they are now the most numerous vertebrates on Earth. Why have birds been so successful? What traits allowed them to increase and diversify so rapidly?.
--------------------------
--------------------------
Seed plants called angiosperms produce seeds in the ovaries of flowers.
--------------------------


## Evaluation of RAGs using Ragas

### Load the Synthetic Dataset for Evaluating the RAGs

In [23]:
from datasets import Dataset

dataset = Dataset.load_from_disk(".content/rag_sciq_data.hf")
print(f"Dataset contains {len(dataset)} rows")

Dataset contains 393 rows


In [25]:
dataset = dataset.select(range(50))

## Ragas 
You need the following columns for Ragas Evaluation
- question: list[str] - These are the questions your RAG pipeline will be evaluated on.
- answer: list[str] - The answer generated from the RAG pipeline and given to the user.
- contexts: list[list[str]] - The contexts that were passed into the LLM to answer the question.
- ground_truths: list[list[str]] - The ground truth answer to the questions. (only required if you are using context_recall)


## Ragas Metrics:
The harmonic mean of these 4 aspects gives you the ragas score which is a single measure of the performance of your QA system across all the important aspects.
- faithfulness - the factual consistency of the answer to the context base on the question.
- context_precision - a measure of how relevant the retrieved context is to the question. Conveys quality of the retrieval pipeline.
- answer_relevancy - a measure of how relevant the answer is to the question
- context_recall: measures the ability of the retriever to retrieve all the necessary information needed to answer the question.

In [26]:
from llama_index import ServiceContext
from llama_index.evaluation import CorrectnessEvaluator
from llama_index.llms import OpenAI
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)

In [27]:
from statistics import harmonic_mean, mean

import nest_asyncio
from llama_index.embeddings import OpenAIEmbedding
from ragas import evaluate
from tqdm import tqdm

nest_asyncio.apply()


def run_eval(embed_model=None, llm_model=None, dimensions=None):
    questions = []
    answers = []
    contexts = []
    ground_truths = []

    correctness_scores = []

    if embed_model:
        if dimensions:
            embedding_model = OpenAIEmbedding(model=embed_model, dimensions=dimensions)
        else:
            embedding_model = OpenAIEmbedding(model=embed_model)
        if llm_model:
            the_service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=OpenAI(model=llm_model))
        else:
            the_service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=OpenAI())
    else:
        if llm_model:
            the_service_context = ServiceContext.from_defaults(llm=OpenAI(model=llm_model))
        else:
            the_service_context = ServiceContext.from_defaults(llm=OpenAI())

    documents = SimpleDirectoryReader(input_files=input_files).load_data("*.txt")
    index = VectorStoreIndex.from_documents(documents, service_context=the_service_context)
    query_engine = index.as_query_engine()

    service_context = ServiceContext.from_defaults(llm=OpenAI())
    evaluator = CorrectnessEvaluator(service_context=service_context)

    for row in tqdm(dataset):
        # The Question
        question = row["query"]
        questions.append(question)

        # The Answer
        response = query_engine.query(question)
        answer = str(response)
        answers.append(answer)

        # Contexts
        context = []
        for node in response.source_nodes:
            context.append(str(node.text))
        contexts.append(context)

        # Ground Truth
        actual_answer = row["reference_answer"]
        ground_truths.append([actual_answer])

        # Correctness with llama-index
        correctness = evaluator.evaluate(
            query=question,
            response=answer,
            reference=actual_answer,
        )
        correctness_scores.append(correctness.score)

    eval_dataset = Dataset.from_dict(
        {"question": questions, "contexts": contexts, "answer": answers, "ground_truths": ground_truths}
    )

    result = evaluate(
        eval_dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
        ],
    )
    baseline_ragas = result
    ragas_score = harmonic_mean(list(result.values()))

    return mean(correctness_scores), baseline_ragas, ragas_score

In [28]:
baseline_correctness, baseline_ragas, baseline_score = run_eval(
    embed_model="text-embedding-ada-002", llm_model="gpt-3.5-turbo"
)
print("Baseline Correctness Score:", baseline_correctness)
print("Baseline Ragas Scores:", baseline_ragas)
print("Baseline Overall Ragas Scores:", baseline_score)

100%|██████████| 50/50 [02:07<00:00,  2.55s/it]


evaluating with [context_precision]


100%|██████████| 4/4 [00:06<00:00,  1.75s/it]


evaluating with [faithfulness]


100%|██████████| 4/4 [00:30<00:00,  7.70s/it]


evaluating with [answer_relevancy]


100%|██████████| 4/4 [00:22<00:00,  5.58s/it]


evaluating with [context_recall]


100%|██████████| 4/4 [00:19<00:00,  4.94s/it]


Baseline Correctness Score: 4.54
Baseline Ragas Scores: {'context_precision': 0.6200, 'faithfulness': 0.9225, 'answer_relevancy': 0.9529, 'context_recall': 0.9840}
Baseline Overall Ragas Scores: 0.8398864762802708


In [29]:
small_35t_correctness, small_35t_ragas, small_35t_score = run_eval(
    embed_model="text-embedding-3-small", llm_model="gpt-3.5-turbo"
)
print("Text Embedding 3 small + GPT 3.5 Turbo Small Correctness Score:", small_35t_correctness)
print("Text Embedding 3 small + GPT 3.5 Turbo Small Ragas Score:", small_35t_ragas)
print("Text Embedding 3 small + GPT 3.5 Turbo Overall Ragas Score:", small_35t_score)

100%|██████████| 50/50 [02:05<00:00,  2.50s/it]


evaluating with [context_precision]


100%|██████████| 4/4 [00:07<00:00,  1.91s/it]


evaluating with [faithfulness]


100%|██████████| 4/4 [00:26<00:00,  6.63s/it]


evaluating with [answer_relevancy]


100%|██████████| 4/4 [00:19<00:00,  4.86s/it]


evaluating with [context_recall]


100%|██████████| 4/4 [00:19<00:00,  4.83s/it]


Text Embedding 3 small + GPT 3.5 Turbo Small Correctness Score: 4.51
Text Embedding 3 small + GPT 3.5 Turbo Small Ragas Score: {'context_precision': 0.6200, 'faithfulness': 0.9260, 'answer_relevancy': 0.9380, 'context_recall': 0.9733}
Text Embedding 3 small + GPT 3.5 Turbo Overall Ragas Score: 0.8357240671153674


In [30]:
large_35t_correctness, large_35t_ragas, large_35t_score = run_eval(
    embed_model="text-embedding-3-large", llm_model="gpt-3.5-turbo"
)
print("Text Embedding 3 Large Correctness Score:", large_35t_correctness)
print("Text Embedding 3 Large Ragas Score:", large_35t_ragas)
print("Text Embedding 3 Large Overall Ragas Score:", large_35t_score)

100%|██████████| 50/50 [02:07<00:00,  2.54s/it]


evaluating with [context_precision]


100%|██████████| 4/4 [00:06<00:00,  1.53s/it]


evaluating with [faithfulness]


100%|██████████| 4/4 [00:29<00:00,  7.37s/it]


evaluating with [answer_relevancy]


100%|██████████| 4/4 [00:19<00:00,  4.98s/it]


evaluating with [context_recall]


100%|██████████| 4/4 [00:20<00:00,  5.12s/it]


Text Embedding 3 Large Correctness Score: 4.45
Text Embedding 3 Large Ragas Score: {'context_precision': 0.6200, 'faithfulness': 0.8808, 'answer_relevancy': 0.9573, 'context_recall': 0.9787}
Text Embedding 3 Large Overall Ragas Score: 0.83079990931469


In [31]:
small_4_correctness, small_4_ragas, small_4_score = run_eval(embed_model="text-embedding-3-small", llm_model="gpt-4")
print("Text Embedding 3 + GPT 4 Correctness Score:", small_4_correctness)
print("Text Embedding 3 + GPT 4 Ragas Score:", small_4_ragas)
print("Text Embedding 3 + GPT 4 Overall Ragas Score:", small_4_score)

100%|██████████| 50/50 [04:18<00:00,  5.17s/it]


evaluating with [context_precision]


100%|██████████| 4/4 [00:05<00:00,  1.49s/it]


evaluating with [faithfulness]


100%|██████████| 4/4 [00:26<00:00,  6.54s/it]


evaluating with [answer_relevancy]


100%|██████████| 4/4 [00:18<00:00,  4.67s/it]


evaluating with [context_recall]


100%|██████████| 4/4 [00:21<00:00,  5.27s/it]


Text Embedding 3 + GPT 4 Correctness Score: 4.23
Text Embedding 3 + GPT 4 Ragas Score: {'context_precision': 0.6000, 'faithfulness': 0.9023, 'answer_relevancy': 0.9357, 'context_recall': 0.9773}
Text Embedding 3 + GPT 4 Overall Ragas Score: 0.821890422345767


In [32]:
small_4t_correctness, small_4t_ragas, small_4t_score = run_eval(
    embed_model="text-embedding-3-small", llm_model="gpt-4-turbo-preview"
)
print("Text Embedding 3 + GPT 4 Turbo Small Correctness Score:", small_4t_correctness)
print("Text Embedding 3 + GPT 4 Turbo Small Ragas Score:", small_4t_ragas)
print("Text Embedding 3 + GPT 4 Turbo Ragas Score:", small_4t_score)

100%|██████████| 50/50 [04:25<00:00,  5.31s/it]


evaluating with [context_precision]


100%|██████████| 4/4 [00:05<00:00,  1.43s/it]


evaluating with [faithfulness]


100%|██████████| 4/4 [00:29<00:00,  7.40s/it]


evaluating with [answer_relevancy]


100%|██████████| 4/4 [00:18<00:00,  4.61s/it]


evaluating with [context_recall]


100%|██████████| 4/4 [00:16<00:00,  4.19s/it]


Text Embedding 3 + GPT 4 Turbo Small Correctness Score: 4.43
Text Embedding 3 + GPT 4 Turbo Small Ragas Score: {'context_precision': 0.6000, 'faithfulness': 0.9537, 'answer_relevancy': 0.9473, 'context_recall': 0.9733}
Text Embedding 3 + GPT 4 Turbo Ragas Score: 0.8336257822994847


In [33]:
large_4t_correctness, large_4t_ragas, large_4t_score = run_eval(
    embed_model="text-embedding-3-large", llm_model="gpt-4-turbo-preview"
)
print("Text Embedding 3 Large + GPT 4 Turbo Small Correctness Score:", large_4t_correctness)
print("Text Embedding 3 Large + GPT 4 Turbo Small Ragas Score:", large_4t_ragas)
print("Text Embedding 3 Large + GPT 4 Turbo Ragas Score:", large_4t_score)

100%|██████████| 50/50 [04:40<00:00,  5.62s/it]


evaluating with [context_precision]


100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


evaluating with [faithfulness]


100%|██████████| 4/4 [00:31<00:00,  7.80s/it]


evaluating with [answer_relevancy]


100%|██████████| 4/4 [00:18<00:00,  4.65s/it]


evaluating with [context_recall]


100%|██████████| 4/4 [00:16<00:00,  4.23s/it]


Text Embedding 3 Large + GPT 4 Turbo Small Correctness Score: 4.42
Text Embedding 3 Large + GPT 4 Turbo Small Ragas Score: {'context_precision': 0.6200, 'faithfulness': 0.9227, 'answer_relevancy': 0.9539, 'context_recall': 0.9920}
Text Embedding 3 Large + GPT 4 Turbo Ragas Score: 0.8415699191725587


In [34]:
large_256_4t_correctness, large_256_4t_ragas, large_256_4t_score = run_eval(
    embed_model="text-embedding-3-small", dimensions=256, llm_model="gpt-4-turbo-preview"
)
print("Text Embedding 3 Large (256) + GPT 4 Turbo Small Correctness Score:", large_256_4t_correctness)
print("Text Embedding 3 Large (256) + GPT 4 Turbo Small Ragas Score:", large_256_4t_ragas)
print("Text Embedding 3 Large (256) + GPT 4 Turbo Ragas Score:", large_256_4t_score)

Loading files: 100%|██████████| 50/50 [00:00<00:00, 1435.75file/s]


100%|██████████| 50/50 [04:35<00:00,  5.51s/it]


evaluating with [context_precision]


100%|██████████| 4/4 [00:05<00:00,  1.41s/it]


evaluating with [faithfulness]


...

evaluating with [answer_relevancy]


100%|██████████| 4/4 [00:20<00:00,  5.20s/it]


evaluating with [context_recall]


100%|██████████| 4/4 [00:17<00:00,  4.42s/it]


Text Embedding 3 Large (256) + GPT 4 Turbo Small Correctness Score: 4.48
Text Embedding 3 Large (256) + GPT 4 Turbo Small Ragas Score: {'context_precision': 0.5900, 'faithfulness': 0.9238, 'answer_relevancy': 0.9432, 'context_recall': 0.9840}
Text Embedding 3 Large (256) + GPT 4 Turbo Ragas Score: 0.8240838436698551


In [35]:
import pandas as pd

In [36]:
comparison = {
    "Embedding Model": [
        "text-embedding-ada-002",
        "text-embedding-3-small",
        "text-embedding-3-large",
        "text-embedding-3-small",
        "text-embedding-3-small",
        "text-embedding-3-large",
    ],
    "LLM Model": [
        "gpt-3.5-turbo",
        "gpt-3.5-turbo",
        "gpt-3.5-turbo",
        "gpt-4",
        "gpt-4-turbo-preview",
        "gpt-4-turbo-preview",
    ],
    "Correctness": [
        baseline_correctness,
        small_35t_correctness,
        large_35t_correctness,
        small_4_correctness,
        small_4t_correctness,
        large_4t_correctness,
    ],
    "Ragas Score": [baseline_score, small_35t_score, large_35t_score, small_4_score, small_4t_score, large_4t_score],
}
df = pd.DataFrame.from_dict(comparison)
df.head(n=10)

,Embedding Model,LLM Model,Correctness,Ragas Score
0,text-embedding-ada-002,gpt-3.5-turbo,4.54,0.839886
1,text-embedding-3-small,gpt-3.5-turbo,4.51,0.835724
2,text-embedding-3-large,gpt-3.5-turbo,4.45,0.830800
3,text-embedding-3-small,gpt-4,4.23,0.821890
4,text-embedding-3-small,gpt-4-turbo-preview,4.43,0.833626
5,text-embedding-3-large,gpt-4-turbo-preview,4.42,0.841570


## Building a Dataset for Fine Tuning

In [54]:
from datasets import Dataset
from openai import OpenAI

dataset = Dataset.load_from_disk(".content/rag_sciq_data.hf")
print(f"Dataset contains {len(dataset)} rows")

questions = []
answers = []
contexts = []
actual = []
updated = []

client = OpenAI()


for row in tqdm(dataset):
    # The Question
    question = row["query"]
    questions.append(question)

    # The Answer
    response = query_engine.query(question)
    answer = str(response)
    answers.append(answer)

    # Contexts
    context = []
    for node in response.source_nodes:
        context.append(str(node.text))
    contexts.append(context)

    # Ground Truth
    actual_answer = row["reference_answer"]
    actual.append([actual_answer])

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": f"""\
For the question and answer below, rewrite the answer to keep it short and to the point.
Example:
  Question: What are the SI units for speed?
  Answer: The SI units for speed are m/s
  Retritten Answer: m/s

Now rewrite the answer for the question below:
  Question:{question}
  Answer:{answer}

ONLY return the rephrased answer. Do not add extra context.""",
            },
        ],
    )
    updated.append(response.choices[0].message.content)
    print(f"Question: {question}\nAnswer: {answer}\nUpdated: {response.choices[0].message.content}\n")
fine_tuning_dataset = Dataset.from_dict(
    {"question": questions, "contexts": contexts, "answer": answers, "actual": actual, "updated": updated}
)

Dataset contains 393 rows


  0%|          | 0/393 [00:00<?, ?it/s]

  0%|          | 1/393 [00:02<13:51,  2.12s/it]

Question: What are some characteristics of birds that distinguish them from other vertebrates?
Answer: Birds have several characteristics that distinguish them from other vertebrates. One key characteristic is that they are endothermic, meaning they can regulate their body temperature internally. Additionally, birds are bipedal, meaning they walk on two legs. Another distinguishing feature is that birds lay amniotic eggs with hard, calcium carbonate shells. These traits, along with others, have contributed to the success and diversification of birds as a class of vertebrates.
Updated: Birds have several distinguishing characteristics such as being endothermic, bipedal, and laying amniotic eggs with hard shells.



  1%|          | 2/393 [00:03<11:05,  1.70s/it]

...

Question: What is the role of enzymes in organisms?
Answer: Enzymes play a crucial role in organisms by facilitating biochemical reactions. They are involved in a large number of reactions, allowing them to occur at the necessary rate for life. Enzymes are responsible for various functions, including the breakdown of food and the conversion of food into energy. Overall, enzymes are essential for the healthy functioning of the body.
Updated: Enzymes facilitate biochemical reactions and are crucial for the healthy functioning of organisms.



100%|█████████▉| 392/393 [22:13<00:01,  1.23s/it]

Question: Can you provide an example of a biochemical reaction that is catalyzed by enzymes?
Answer: An example of a biochemical reaction that is catalyzed by enzymes is the breakdown of food and its conversion to energy.
Updated: The breakdown of food and its conversion to energy.



100%|██████████| 393/393 [22:16<00:00,  3.40s/it]

Question: How do enzymes allow reactions to occur at the necessary rate for life?
Answer: Enzymes allow reactions to occur at the necessary rate for life by catalyzing biochemical reactions. They speed up the rate of these reactions by lowering the activation energy required for the reaction to take place. This allows the reactions to occur more quickly and efficiently, enabling the various processes necessary for life to proceed at the appropriate pace.
Updated: Enzymes speed up biochemical reactions by lowering the activation energy required. This allows reactions to occur more quickly, enabling necessary processes for life.



In [55]:
documents_folder = os.path.join(os.path.abspath(""), ".content")
fine_tuning_dataset.save_to_disk(documents_folder + "/rag_fine_tuning_dataset.hf")

Saving the dataset (1/1 shards): 100%|██████████| 393/393 [00:00<00:00, 46333.52 examples/s]


In [56]:
fine_tuning_dataset.to_pandas().head()

,question,contexts,answer,actual,updated
0,What are some characteristics of birds that di...,[Birds are endothermic tetrapod vertebrates. T...,Birds have several characteristics that distin...,[Some characteristics of birds that distinguis...,Birds have several distinguishing characterist...
1,"How do birds reproduce, and what is unique abo...",[Birds are endothermic tetrapod vertebrates. T...,Birds reproduce through the process of laying ...,[Birds reproduce by laying amniotic eggs. Thes...,"Birds reproduce by laying amniotic eggs, which..."
2,Why are birds considered the most numerous ver...,[Birds are endothermic tetrapod vertebrates. T...,Birds are considered the most numerous vertebr...,[Birds are considered the most numerous verteb...,Birds are the most numerous vertebrates on Ear...
3,What factors have contributed to the rapid inc...,[Birds are endothermic tetrapod vertebrates. T...,"Factors such as bipedal locomotion, the abilit...",[The traits that have contributed to the rapid...,"Bipedal locomotion, amniotic eggs with hard sh..."
4,"How do birds move, and what is the significanc...",[Birds are endothermic tetrapod vertebrates. T...,"Birds move by walking on two legs, which is kn...","[Birds move by walking or hopping on two legs,...","Birds move by walking on two legs, known as bi..."
